In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
import torch
torch.cuda.is_available()

True

# 10.7 文本情感分类：使用循环神经网络

In [21]:
import collections
import os
import random
import tarfile
import torch
from torch import nn
import torchtext
# from torchtext.vocab import vocab as Vocab
import torchtext.vocab as Vocab
import torch.utils.data as Data

from nltk.tokenize import word_tokenize

import sys
sys.path.append("..")

# import d2lzh_pytorch as d2l

# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(torch.__version__, device)
# print(torch.__version__)

1.9.0+cu102 cuda


## 10.7.1 文本情感分类数据
### 10.7.1.1 读取数据

In [5]:
BASE = os.getcwd()
os.listdir(BASE)

['.config', 'sample_data', 'drive']

In [6]:
# datasets_root_path = "/Users/lusong/Datasets/"
datasets_root_path = os.path.join(BASE, "sample_data")
os.listdir(datasets_root_path)

['README.md',
 'anscombe.json',
 'aclImdb',
 '.ipynb_checkpoints',
 'mnist_test.csv',
 'mnist_train_small.csv',
 'california_housing_train.csv',
 'california_housing_test.csv']

In [7]:
# DATA_ROOT = os.path.join(datasets_root_path, 'Dive-into-DL-PyTorch', "data")
DATA_ROOT = datasets_root_path

In [8]:
os.listdir(DATA_ROOT)

['README.md',
 'anscombe.json',
 'aclImdb',
 '.ipynb_checkpoints',
 'mnist_test.csv',
 'mnist_train_small.csv',
 'california_housing_train.csv',
 'california_housing_test.csv']

In [25]:
# fname = os.path.join(DATA_ROOT, "aclImdb_v1.tar.gz")
# if not os.path.exists(os.path.join(DATA_ROOT, "aclImdb")):
#     print("从压缩包解压...")
#     with tarfile.open(fname, 'r') as f:
#         f.extractall(DATA_ROOT)

!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar xfz aclImdb_v1.tar.gz

# import requests

# url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
filename = os.path.join(DATA_ROOT, 'aclImdb_v1.tar.gz')
# with open(filename, "wb") as f:
#     r = requests.get(url)
#     f.write(r.content)

--2021-09-02 02:46:24--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  43.2MB/s    in 1.9s    

2021-09-02 02:46:26 (43.2 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [17]:
if not os.path.exists(filename):
  print("从压缩包解压...")
  with tarfile.open(filename, 'r') as f:
      f.extractall(DATA_ROOT)

In [13]:
from tqdm import tqdm
def read_imdb(folder='train', data_root=os.path.join(DATA_ROOT, "aclImdb")):  # 本函数已保存在d2lzh_pytorch包中方便以后使用
    data = []
    for label in ['pos', 'neg']:
        folder_name = os.path.join(data_root, folder, label)
        for file in tqdm(os.listdir(folder_name)):
            with open(os.path.join(folder_name, file), 'rb') as f:
                review = f.read().decode('utf-8').replace('\n', '').lower()
                data.append([review, 1 if label == 'pos' else 0])
    random.shuffle(data)
    return data

data_root = os.path.join(DATA_ROOT, "aclImdb")
train_data, test_data = read_imdb('train', data_root), read_imdb('test', data_root)

100%|██████████| 12500/12500 [00:00<00:00, 32362.48it/s]


In [14]:
type(train_data), type(test_data), len(train_data), len(test_data)

(list, list, 25000, 25000)

In [15]:
train_data[0], test_data[0]

(["this version of bleak house is the best adaptation of a classic novel known to me. the representation of the court of chancery as a 'character' in the drama is magnificent. the acting is marvellous, from the sinister tulkinghorn, to the dedlocks, smallweed, crooke, miss flyte, and the two young lovers. but it is the spider's web of chancery that holds the whole thing together, and the cinematography is superb. what mistake did the bbc make about copyright that meant that this version could not be seen in the uk on either video or dvd for many years? i tried to find out from them, but faced a stone wall. in the end i got a dvd copy from canada.",
  1],
 ['ok. is barney the best children\'s show of all time? of course not. but in some of the comments left by other members of imdb you would think it was a multi million dollar production with high class actors and a ridiculous budget for special effects. well guess what? it\'s barney for god\'s sake. he shows children good behavior, goo

### 10.7.1.2 预处理数据

In [24]:
# def get_tokenized_imdb(data):  # 本函数已保存在d2lzh_pytorch包中方便以后使用
#     """
#     data: list of [string, label]
#     """
#     def tokenizer(text):
#         return [tok for tok in text.lower().split(' ')]
#     return [tokenizer(review) for review, _ in data]

import nltk
nltk.download('punkt')

def get_tokenized_imdb(data):
    """
    data: list of [string, label]
    """
    return [word_tokenize(review) for review, _ in data]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [23]:
tokenized_data = get_tokenized_imdb(train_data)
len(tokenized_data)

LookupError: ignored

In [20]:
tokenized_data[0]

['this',
 'version',
 'of',
 'bleak',
 'house',
 'is',
 'the',
 'best',
 'adaptation',
 'of',
 'a',
 'classic',
 'novel',
 'known',
 'to',
 'me.',
 'the',
 'representation',
 'of',
 'the',
 'court',
 'of',
 'chancery',
 'as',
 'a',
 "'character'",
 'in',
 'the',
 'drama',
 'is',
 'magnificent.',
 'the',
 'acting',
 'is',
 'marvellous,',
 'from',
 'the',
 'sinister',
 'tulkinghorn,',
 'to',
 'the',
 'dedlocks,',
 'smallweed,',
 'crooke,',
 'miss',
 'flyte,',
 'and',
 'the',
 'two',
 'young',
 'lovers.',
 'but',
 'it',
 'is',
 'the',
 "spider's",
 'web',
 'of',
 'chancery',
 'that',
 'holds',
 'the',
 'whole',
 'thing',
 'together,',
 'and',
 'the',
 'cinematography',
 'is',
 'superb.',
 'what',
 'mistake',
 'did',
 'the',
 'bbc',
 'make',
 'about',
 'copyright',
 'that',
 'meant',
 'that',
 'this',
 'version',
 'could',
 'not',
 'be',
 'seen',
 'in',
 'the',
 'uk',
 'on',
 'either',
 'video',
 'or',
 'dvd',
 'for',
 'many',
 'years?',
 'i',
 'tried',
 'to',
 'find',
 'out',
 'from',
 't

In [ ]:
counter = collections.Counter([tk for st in tokenized_data for tk in st])

In [ ]:
# counter
sorted_by_freq_tuples = sorted(counter.items(), key=lambda x: x[1], reverse=True)

In [ ]:
# sorted_by_freq_tuples

In [ ]:
ordered_dict = collections.OrderedDict(sorted_by_freq_tuples)

In [ ]:
ordered_dict

OrderedDict([('the', 322174),
             ('a', 159949),
             ('and', 158556),
             ('of', 144459),
             ('to', 133965),
             ('is', 104170),
             ('in', 90521),
             ('i', 70477),
             ('this', 69711),
             ('that', 66287),
             ('it', 65489),
             ('/><br', 50935),
             ('was', 47023),
             ('as', 45098),
             ('for', 42839),
             ('with', 42725),
             ('but', 39757),
             ('on', 31618),
             ('movie', 30883),
             ('his', 29058),
             ('are', 28742),
             ('not', 28591),
             ('film', 27774),
             ('you', 27558),
             ('have', 27343),
             ('he', 26175),
             ('be', 25690),
             ('at', 22730),
             ('one', 22477),
             ('by', 21974),
             ('an', 21239),
             ('they', 20623),
             ('from', 19933),
             ('all', 19738),
             

In [ ]:
# v1 = Vocab.Vocab(ordered_dict, min_freq=5)
v1 = Vocab.vocab(counter, min_freq=5)

In [ ]:
v1.get_itos()

['i',
 'saw',
 'this',
 'film',
 'without',
 'to',
 'know',
 'what',
 'about',
 "i'm",
 'a',
 'fan',
 'of',
 'branagh,',
 'even',
 'more',
 'his',
 'films,',
 'and,',
 'in',
 'the',
 'beginning,',
 'it',
 'only',
 'for',
 'this...',
 'and',
 'finished',
 'with',
 'tears',
 'my',
 'eyes,',
 'because',
 'great,',
 'great',
 'values,',
 'affect',
 'brave',
 'philosophy',
 'life',
 'girl.',
 'recommended',
 'people',
 'who',
 'are',
 'bored',
 'tv',
 'programming',
 '(in',
 'spain,',
 'at',
 'least).',
 'so',
 'often',
 'band',
 'will',
 'get',
 'together',
 'concert',
 'find',
 'that',
 'they',
 'just',
 "can't",
 'together.',
 'not',
 'here.',
 'is',
 'shear',
 'brilliance',
 'from',
 'start',
 'finish.',
 'these',
 'three',
 'musicians',
 'obviously',
 'got',
 'beforehand',
 'plotted',
 'planned',
 'was',
 'needed',
 'ensure',
 'nostalgic',
 'bash',
 'satisfy',
 "someone's",
 'ego.',
 'obvious',
 'start,',
 'before',
 'step',
 'on',
 'stage.',
 'many',
 'faces',
 'crowd',
 "weren't",
 '

In [ ]:
def get_vocab_imdb(data):  # 本函数已保存在d2lzh_pytorch包中方便以后使用
    tokenized_data = get_tokenized_imdb(data)
    counter = collections.Counter([tk for st in tokenized_data for tk in st])
    sorted_by_freq_tuples = sorted(counter.items(), key=lambda x: x[1], reverse=True)
    ordered_dict = collections.OrderedDict(sorted_by_freq_tuples)
    return Vocab(ordered_dict, min_freq=1)

vocab = get_vocab_imdb(train_data)
'# words in vocab:', len(vocab)

('# words in vocab:', 252192)

In [ ]:
dir(vocab)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__jit_unused_properties__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__prepare_scriptable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_backward_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',


In [ ]:
vocab.get_itos()[0]

'the'

In [ ]:
vocab["a"]

1

In [ ]:
def preprocess_imdb(data, vocab):  # 本函数已保存在d2lzh_torch包中方便以后使用
    max_l = 500  # 将每条评论通过截断或者补0，使得长度变成500

    def pad(x):
        return x[:max_l] if len(x) > max_l else x + [0] * (max_l - len(x))

    tokenized_data = get_tokenized_imdb(data)
    features = torch.tensor([pad([vocab[word] for word in words]) for words in tokenized_data])
    labels = torch.tensor([score for _, score in data])
    return features, labels

In [ ]:
processed_train_data = preprocess_imdb(train_data, vocab)

In [ ]:
processed_test_data = preprocess_imdb(test_data, vocab)

RuntimeError: Token not-so-great. not found and default index is not set

### 10.7.1.3 创建数据迭代器

In [ ]:
batch_size = 64
train_set = Data.TensorDataset(*preprocess_imdb(train_data, vocab))
test_set = Data.TensorDataset(*preprocess_imdb(test_data, vocab))

train_iter = Data.DataLoader(train_set, batch_size, shuffle=True)
test_iter = Data.DataLoader(test_set, batch_size)

In [ ]:
for X, y in train_iter:
    print('X', X.shape, 'y', y.shape)
    break
'#batches:', len(train_iter)

X torch.Size([64, 500]) y torch.Size([64])


('#batches:', 391)

## 10.7.2 使用循环神经网络的模型

In [ ]:
class BiRNN(nn.Module):
    def __init__(self, vocab, embed_size, num_hiddens, num_layers):
        super(BiRNN, self).__init__()
        self.embedding = nn.Embedding(len(vocab), embed_size)
        
        # bidirectional设为True即得到双向循环神经网络
        self.encoder = nn.LSTM(input_size=embed_size, 
                                hidden_size=num_hiddens, 
                                num_layers=num_layers,
                                bidirectional=True)
        self.decoder = nn.Linear(4*num_hiddens, 2) # 初始时间步和最终时间步的隐藏状态作为全连接层输入

    def forward(self, inputs):
        # inputs的形状是(批量大小, 词数)，因为LSTM需要将序列长度(seq_len)作为第一维，所以将输入转置后
        # 再提取词特征，输出形状为(词数, 批量大小, 词向量维度)
        embeddings = self.embedding(inputs.permute(1, 0))
        # rnn.LSTM只传入输入embeddings，因此只返回最后一层的隐藏层在各时间步的隐藏状态。
        # outputs形状是(词数, 批量大小, 2 * 隐藏单元个数)
        outputs, _ = self.encoder(embeddings) # output, (h, c)
        # 连结初始时间步和最终时间步的隐藏状态作为全连接层输入。它的形状为
        # (批量大小, 4 * 隐藏单元个数)。
        encoding = torch.cat((outputs[0], outputs[-1]), -1)
        outs = self.decoder(encoding)
        return outs

In [ ]:
embed_size, num_hiddens, num_layers = 100, 100, 2
net = BiRNN(vocab, embed_size, num_hiddens, num_layers)

### 10.7.2.1 加载预训练的词向量

In [ ]:
glove_vocab = Vocab.GloVe(name='6B', dim=100, cache=os.path.join(DATA_ROOT, "glove"))

In [ ]:
dir(glove_vocab)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'cache',
 'dim',
 'get_vecs_by_tokens',
 'itos',
 'stoi',
 'unk_init',
 'url',
 'vectors']

In [ ]:
glove_vocab.vectors.shape

torch.Size([400000, 100])

In [ ]:
glove_vocab.vectors[0].shape[0]

100

In [ ]:
len(vocab)

252192

In [ ]:
def load_pretrained_embedding(words, pretrained_vocab):
    """从预训练好的vocab中提取出words对应的词向量"""
    embed = torch.zeros(len(words), pretrained_vocab.vectors[0].shape[0]) # 初始化为0
    oov_count = 0 # out of vocabulary
    for i, word in enumerate(words):
        try:
            idx = pretrained_vocab.stoi[word]
            embed[i, :] = pretrained_vocab.vectors[idx]
        except KeyError:
            oov_count += 1
    if oov_count > 0:
        print("There are %d oov words." % oov_count)
    return embed

In [ ]:
net.embedding.weight.data.copy_(load_pretrained_embedding(vocab.itos, glove_vocab))
net.embedding.weight.requires_grad = False # 直接加载预训练好的, 所以不需要更新它

There are 21202 oov words.


### 10.7.2.2 训练并评价模型

In [ ]:
lr, num_epochs = 0.01, 5
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)
loss = nn.CrossEntropyLoss()
d2l.train(train_iter, test_iter, net, loss, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 0.5415, train acc 0.719, test acc 0.819, time 48.7 sec
epoch 2, loss 0.1897, train acc 0.837, test acc 0.852, time 53.0 sec
epoch 3, loss 0.1105, train acc 0.857, test acc 0.844, time 51.6 sec
epoch 4, loss 0.0719, train acc 0.881, test acc 0.865, time 52.1 sec
epoch 5, loss 0.0519, train acc 0.894, test acc 0.852, time 51.2 sec


In [ ]:
# 本函数已保存在d2lzh包中方便以后使用
def predict_sentiment(net, vocab, sentence):
    """sentence是词语的列表"""
    device = list(net.parameters())[0].device
    sentence = torch.tensor([vocab.stoi[word] for word in sentence], device=device)
    label = torch.argmax(net(sentence.view((1, -1))), dim=1)
    return 'positive' if label.item() == 1 else 'negative'

In [ ]:
predict_sentiment(net, vocab, ['this', 'movie', 'is', 'so', 'great'])

'positive'

In [ ]:
predict_sentiment(net, vocab, ['this', 'movie', 'is', 'so', 'bad'])

'negative'